In [ ]:
import wandb
import os
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.font_manager as fm
import scipy
import tree
import json
import glob
fm.fontManager.addfont('../times.ttf')
fm.fontManager.addfont('../times_Italic.ttf')

jobs = {
    1: [
        "skyworkrm_dpo_online_0920T16:13",
        "skyworkrm_dpo_online_0920T15:15",
        "skyworkrm_dpo_online_0917T08:29",
    ], # online dpo
    2: [
        "skyworkrm_dpo_duelingts_0925T23:05",
        "skyworkrm_dpo_duelingts_0925T20:17",
        "skyworkrm_dpo_duelingts_0925T12:39"
    ], # sea-dpo without MAP
    3: [
        "skyworkrm_dpo_dueling_dyna_0926T14:07",
        "skyworkrm_dpo_dueling_dyna_0926T09:47",
        "skyworkrm_dpo_dueling_dyna_0926T05:24",
    ], # sea-dpo
    4: [
        
    ],
    5: [],
    6: [],
    7: [],
}

api = wandb.Api()
save_dir = "./gpt_oracle_data"
runs = api.runs("lkevinzc/ellm_simpo")
data_list = []
cfg_list = []

features_of_interest = ["eval/rm_win_rate_prob", "misc/query_step"]
flatten_fns = tree.flatten(jobs)
os.makedirs(save_dir, exist_ok=True)
for run in tqdm.tqdm(runs):
    cfg = run.config
    if run.name in flatten_fns:
        print(run.name)
        name = run.name
        if not os.path.exists(f"{save_dir}/{name}.csv"):
            data = run.history(keys=features_of_interest)
            data["seed"] = cfg["seed"]
            data["run_name"] = cfg["wandb_run_name"]
            
            if "online" in cfg["wandb_run_name"]:
                method = "online"
            elif "offline" in cfg["wandb_run_name"]:
                method = "offline"
            elif "sea" in cfg["wandb_run_name"]:
                method = "sea"
            elif "apl" in cfg["wandb_run_name"]:
                method = "apl"
            elif "xpo" in cfg["wandb_run_name"]:
                method = "xpo"

            data["method"] = method
            data.to_csv(f"{save_dir}/{name}.csv")